In [7]:
import matplotlib
matplotlib.use('Qt4Agg')
from matplotlib import pyplot as plt

import matplotlib as mpl
from sklearn.manifold import MDS
from sklearn.externals import joblib
import numpy as np
import pandas as pd
import time
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
#dir = os.path.dirname(__file__)

file_paths = [r'C:\Users\Danny\PycharmProjects\BobRoss\Transcript Scraper\Transcripts\The Joy of Painting - Season 1\Bob Ross - Autumn Mountain (Season 1 Episode 7)\Bob Ross - Autumn Mountain (Season 1 Episode 7).txt',
               r'C:\Users\Danny\PycharmProjects\BobRoss\Transcript Scraper\Transcripts\The Joy of Painting - Season 1\Bob Ross - A Walk in the Woods (Season 1 Episode 1)\Bob Ross - A Walk in the Woods (Season 1 Episode 1).txt']
file_paths = []
transcripts = []
titles = []
stopwords = nltk.corpus.stopwords.words('english')

# load nltk's SnowballStemmer as variabled 'stemmer'
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
stopwords.extend(['little', 'coat', 'shoot','ol','sorta', 'odorless', 'em', 'wan', 'wan na', 'yeah', 'something'])
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token) and token not in stopwords:
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token) and token not in stopwords:
            filtered_tokens.append(token)
    return filtered_tokens


# not super pythonic, no, not at all.
# use extend so it's a big flat list of vocab
pattern = re.compile('Bob Ross - (.*)\.txt$')
for root, dirs, files in os.walk(r".\Transcripts"):
    for file in files:
        if file.endswith(".txt") and file not in ["data.txt", "paint.txt"]:
            f = os.path.join(root, file)
            file_paths.append(f)
            with open(f, 'rb') as infile:
                text = infile.read().decode('utf8')
                if text:
                    transcripts.append(text)
                    titles.append(pattern.match(file)[1])

totalvocab_stemmed = []
totalvocab_tokenized = []
#for file in file_paths:
 #   with open(file, 'rb') as infile:
  #      text = infile.read().decode('utf8')
   #     transcripts.append(text)

for transcript in transcripts:
    allwords_stemmed = tokenize_and_stem(transcript)  # for each item in transcripts, tokenize/stem
    if allwords_stemmed:
        totalvocab_stemmed.extend(allwords_stemmed)  # extend the 'totalvocab_stemmed' list
    allwords_tokenized = tokenize_only(transcript)
    totalvocab_tokenized.extend(allwords_tokenized)

vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index=totalvocab_stemmed)
print('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')
print(vocab_frame.head())

# gets us a vocab for the entire corpus and then we need to create a Term Freq Inverse Document Frequency Matrx
#       epi_1 -- epi_2
#word1  x           y
#word2  z           u

# use the tfidf to do some clustering, maybe k means
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

joblib.dump(tfidf_vectorizer, 'vectorizer.pkl')
tfidf_vectorizer = joblib.load('vectorizer.pkl')

start = time.time()
tfidf_matrix = tfidf_vectorizer.fit_transform(transcripts) #fit the vectorizer to synopses
end = time.time()
print("Timing was: " + str(end-start))
print(tfidf_matrix.shape)

terms = tfidf_vectorizer.get_feature_names()
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)
print(dist)

there are 747437 items in vocab_frame
     words
hi      hi
im      im
bob    bob
ross  ross
next  next
Timing was: 51.78325128555298
(403, 1194)
[[  3.33066907e-16   6.57155087e-01   5.40618918e-01 ...,   8.05370901e-01
    7.94339575e-01   7.69224081e-01]
 [  6.57155087e-01  -2.22044605e-16   5.93467262e-01 ...,   7.35639282e-01
    6.70042039e-01   6.82503565e-01]
 [  5.40618918e-01   5.93467262e-01   0.00000000e+00 ...,   7.59900013e-01
    7.49788280e-01   7.06075570e-01]
 ..., 
 [  8.05370901e-01   7.35639282e-01   7.59900013e-01 ...,  -2.22044605e-16
    6.44629773e-01   6.58404655e-01]
 [  7.94339575e-01   6.70042039e-01   7.49788280e-01 ...,   6.44629773e-01
    0.00000000e+00   4.51341624e-01]
 [  7.69224081e-01   6.82503565e-01   7.06075570e-01 ...,   6.58404655e-01
    4.51341624e-01  -4.44089210e-16]]


In [8]:
from nltk.tag import pos_tag

def strip_proppers_POS(text):
    tagged = pos_tag(text.split()) #use NLTK's part of speech tagger
    non_propernouns = [word for word,pos in tagged if pos != 'NNP' and pos != 'NNPS']
    return non_propernouns

In [11]:
from gensim import corpora, models, similarities 

#remove proper names - preprocess is a list of lists
#%time preprocess = [strip_proppers_POS(doc) for doc in transcripts]

#tokenize
#print(preprocess[1])
tokenized_text = []
start = time.time()
for text in transcripts:
    tokenized_text.append(tokenize_and_stem(text))
end = time.time()
print("Timing was: " + str(end-start))

#remove stop words
%time texts = [[word for word in text if word not in stopwords] for text in tokenized_text]

Timing was: 45.905484437942505
Wall time: 4.33 s


In [12]:
#create a Gensim dictionary from the texts
dictionary = corpora.Dictionary(texts)

#remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
dictionary.filter_extremes(no_below=1, no_above=0.8)

#convert the dictionary to a bag of words corpus for reference
corpus = [dictionary.doc2bow(text) for text in texts]

In [13]:
# run the LDA model (long wait)
%time lda = models.LdaModel(corpus, num_topics=5, id2word=dictionary, update_every=5, chunksize=10000, passes=100)

Wall time: 12min 4s


In [14]:
lda.show_topics()

[(0,
  '0.028*"snow" + 0.014*"winter" + 0.013*"ok" + 0.011*"scene" + 0.011*"alright" + 0.008*"mountain" + 0.008*"caus" + 0.007*"pprusian" + 0.007*"ole" + 0.007*"phphphthalo"'),
 (1,
  '0.008*"alright" + 0.007*"bush" + 0.006*"gesso" + 0.006*"caus" + 0.006*"doesnt" + 0.006*"corner" + 0.006*"though" + 0.005*"inch" + 0.005*"ochr" + 0.005*"decid"'),
 (2,
  '0.011*"lay" + 0.009*"sap" + 0.008*"tini" + 0.008*"play" + 0.008*"bush" + 0.007*"cloud" + 0.007*"umber" + 0.007*"music" + 0.007*"ok" + 0.007*"shadow"'),
 (3,
  '0.020*"mountain" + 0.010*"cloud" + 0.008*"shadow" + 0.007*"bush" + 0.007*"reflect" + 0.007*"layer" + 0.006*"roll" + 0.006*"decis" + 0.006*"inch" + 0.005*"stroke"'),
 (4,
  '0.035*"barn" + 0.033*"build" + 0.031*"roof" + 0.022*"hous" + 0.022*"door" + 0.018*"window" + 0.016*"board" + 0.016*"fenc" + 0.012*"cabin" + 0.012*"bare"')]

In [15]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)
matrix = [ele[1] for ele in topics_matrix]
topics_matrix = np.array(matrix)

topic_words = topics_matrix[:,:,0]
for i in topic_words:
    print([str(word) for word in i])
    print()

['snow', 'winter', 'ok', 'scene', 'alright', 'mountain', 'caus', 'pprusian', 'ole', 'phphphthalo', 'cold', 'twoinch', 'decid', 'doesnt', 'shadow', 'upward', 'somethin', 'care', 'cloud', 'wash']

['alright', 'bush', 'gesso', 'caus', 'doesnt', 'corner', 'though', 'inch', 'ochr', 'decid', 'sienna', 'liner', 'phphphthalo', 'rock', 'sap', 'clear', 'least', 'stone', 'wave', 'hang']

['lay', 'sap', 'tini', 'play', 'bush', 'cloud', 'umber', 'music', 'ok', 'shadow', 'perman', 'next', 'bare', 'burnt', 'limb', 'scrape', 'reflect', 'almighti', 'mountain', 'week']

['mountain', 'cloud', 'shadow', 'bush', 'reflect', 'layer', 'roll', 'decis', 'inch', 'stroke', 'lay', 'corner', 'tini', 'pressur', 'base', 'bare', 'angl', 'hes', 'least', 'illus']

['barn', 'build', 'roof', 'hous', 'door', 'window', 'board', 'fenc', 'cabin', 'bare', 'path', 'post', 'shingl', 'wood', 'shed', 'stand', 'farmer', 'rail', 'cow', 'porch']



In [16]:
print(type(topics_matrix[0]))

<class 'numpy.ndarray'>
